In [15]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import os

# Data Pipeline

In [16]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [17]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


## Encoding

In [18]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character',
                                                   standardize = 'lower')
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [19]:
encoded

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12], dtype=int64)>

In [20]:
encoded -= 2

In [21]:
n_tokens = text_vec_layer.vocabulary_size()
dataset_size = len(encoded)

In [22]:
dataset_size

1115394

In [23]:
from Utility import to_dataset
length = 100 
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1000000], length = length, shuffle=True, seed = 42)
valid_set = to_dataset(encoded[1000000: 1060000], length = length)
test_set = to_dataset(encoded[1060000:], length = length)

## Model

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = n_tokens, output_dim = 16),
    tf.keras.layers.GRU(128, return_sequences= True),
    tf.keras.layers.Dense(n_tokens, activation = 'softmax')
])

model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Nadam(),
              metrics = ['accuracy'])

In [25]:
if os.path.exists("shakspeare_model.keras"):
    tf.keras.models.load_model("shakspeare_model.keras")
else:

    history = model.fit(train_set , validation_data= valid_set, epochs = 10,
                        callbacks = tf.keras.callbacks.ModelCheckpoint('shakspeare_model.keras', monitor = 'val_accuracy', save_best_only= True))

In [28]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X -2),
    model
])